# Niedersachsen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Niedersachsen is `DE9`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [16]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from typing import Union, Dict, List
from datetime import datetime as dt
from dateparser import parse
import warnings

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('niedersachsen').input_path
BASE

'/home/mirko/Dropbox/python/camelsp/input_data/NiS_Niedersachsen'

## Parse data

Niedersachen produced only one file. I guess this needs to be pivoted.

In [3]:
raw = pd.read_csv(os.path.join(BASE, 'exp-peg-par252.csv'), encoding='latin1', sep=';', decimal=',')
raw

,MESSSTELLE_NR,DATUM,LANGNAME,BEZEICHNUNG,KENNUNG_ID,WERT,EINHEIT
0,3183101,10.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,0.853,m³/s
1,3183101,11.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,0.853,m³/s
2,3183101,12.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,0.853,m³/s
3,3183101,13.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,0.772,m³/s
4,3183101,07.09.87,Sudendorf,Abfluss Tagesmittelwert,NaN,0.938,m³/s
...,...,...,...,...,...,...,...
4195557,3183101,05.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,1.030,m³/s
4195558,3183101,06.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,1.030,m³/s
4195559,3183101,07.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,1.800,m³/s
4195560,3183101,08.01.85,Sudendorf,Abfluss Tagesmittelwert,NaN,1.200,m³/s


In [4]:
# id column is MESSSTELLE_NR
id_column = 'MESSSTELLE_NR'

In [5]:
# How many different variables are there?
names = []
for _, df in raw.groupby(id_column):
    names.extend(df.BEZEICHNUNG.unique().tolist())
set(names)

{'Abfluss Tagesmittelwert'}

In [ ]:
# total messstellen
N = len(raw.groupby(id_column))
print(f"Messstellen: {N}")

Messstellen: 282


New create a list of collected 'metadata' and the actual discharge data. 

Extract  all metadata for this federal state, without using the `Bundesland` context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [12]:
# result container
meta = []
data = []

# group by id
N = len(raw.groupby(id_column))

# go for it
for nr, df in tqdm(raw.groupby(id_column)):
    meta.append({
        id_column: str(nr),
        'BEZEICHNUNG': df.BEZEICHNUNG.unique().tolist(),
        'EINHEIT': df.EINHEIT.unique().tolist(),
        'LANGNAME': df.LANGNAME.unique().tolist(),
        'KENNUNG_ID': df.KENNUNG_ID.unique().tolist()
    })
    data.append(pd.DataFrame({
        'date': [d.date() if isinstance(d, dt) else parse(d) for d in df.DATUM],
        'q': df.WERT,
        'flag': np.NaN
    }))

print(f"Extracted {len(data)} timeseries")
        

  9%|▉         | 26/282 [06:50<1:07:23, 15.80s/it]


KeyboardInterrupt: 

### metadata

Ok, let's get really wild. Check that the code above produced only lists of 1 unique value per group. Otherwise the metadata would change over time for the same Messstelle and that would be a problem

In [17]:
def tidy_metadata(meta: List[dict]) -> pd.DataFrame:
    pmeta = []
    for i, m in enumerate(meta):
        out = {}
        for k, v in m.items():
            if isinstance(v, list):
                if len(v) == 1:
                    out[k] = v[0]
                else:
                    warnings.warn(f"Line {i + 1}: More than one value found for {k}: [{', '.join(v)}]")
            else:
                out[k] = v
        pmeta.append(out)
    return pd.DataFrame(pmeta)


### Finally run

Now, the Q and W data can be extracted along with the metadata. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [20]:
with Bundesland('Niedersachsen') as bl:
    # catch warnings
    with warnings.catch_warnings(record=True) as warns:
        # tidy the metadata
        metadata = tidy_metadata(meta)

        # save the metadata
        bl.save_raw_metadata(metadata, id_column, overwrite=True)

        # for reference, call the nuts-mapping as table
        nuts_map = bl.nuts_table
        print(nuts_map.head())
    
        # go for all ids
        for meta, df in tqdm(zip(meta, data), total=N):
            # get the id
            provider_id = meta[id_column]

            # save
            bl.save_timeseries(df, provider_id)
        
        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DE910000     3183101  ./DE9/DE910000/DE910000_data.csv
1  DE910010     3346103  ./DE9/DE910010/DE910010_data.csv
2  DE910020     3437108  ./DE9/DE910020/DE910020_data.csv
3  DE910030     3445100  ./DE9/DE910030/DE910030_data.csv
4  DE910040     3449100  ./DE9/DE910040/DE910040_data.csv


  9%|▉         | 26/282 [00:01<00:19, 13.44it/s]
